# Parsing csv to xlsx
## Data Sources
https://api.ehealth.gov.ua/api/services
## Commit git
- 3ed8a2e Start
- 2550448b First read json
- 75c0f119c modul json
- efbb4001 del dct
- 87d560a2 write parsing_1.xlsx parsing_2.xlsx


In [17]:
import pandas as pd
import re

df = pd.read_json('./data/processed/services.json', 
                  orient='records')
# конвертація datetime64[ns, UTC] в object ('2020-03-31') без часу
for i in ['inserted_at', 'updated_at']:
    df[i] = pd.to_datetime(df[i]).dt.date

In [20]:
df.to_excel("./data/interim/first.xlsx", index=False)

## Считування за допомогою модуля json

In [1]:
import json
import openpyxl

# считування json файла в пітоновський словник type(data) -> dict
with open('./data/raw/services.json', 'r', encoding='UTF-8') as file:
    dct = json.load(file)

In [2]:
wb = openpyxl.Workbook()
ws = wb.active
ws['A1'] = 'cod'
ws['B1'] = 'description'
row = 2
for i in dct['data']:
    if i['code'][:4] not in ['Клас', 'LAB0']:
        ws.cell(row, column=1).value = i['code']
        ws.cell(row, column=2).value = i['name']
        row += 1
wb.save('./data/interim/parsing_1.xlsx')

In [3]:
wb = openpyxl.Workbook()
ws = wb.active
ws['A1'] = 'cod'
ws['B1'] = 'cod2'
ws['C1'] = 'description'
row = 2
for i in dct['data']:
    if i['code'][:4] not in ['Клас', 'LAB0']:
        for j in i['services']:
            ws.cell(row, column=1).value = i['code']
            ws.cell(row, column=2).value = j['code']
            ws.cell(row, column=3).value = j['name']
            row += 1
wb.save('./data/interim/parsing_2.xlsx')

In [45]:
wb1 = openpyxl.Workbook()
ws1 = wb1.active
ws1['A1'] = 'cod'
ws1['B1'] = 'cod2'
ws1['C1'] = 'cod3'
ws1['D1'] = 'description'

cl = ['Клас 5', 'Клас 14', 'Клас 17', 'Клас 4',
     'Клас 10', 'Клас 8', 'Клас 18', 'Клас 2',
     'Клас 9', 'Клас 15', 'Клас 19', 'Клас 12',
     'Клас 16', 'Клас 3', 'Клас 20', 'Клас 1',
     'Клас 13', 'Клас 11', 'Клас 7', 'LAB000']
row = 2
for i in dct['data']:
    if i['code'] in cl:
        for j in i['groups']:
            for k in j['groups']:
                for h in k['services']:
                    ws1.cell(row, column=1).value = i['code']
                    ws1.cell(row, column=2).value = j['code']
                    ws1.cell(row, column=3).value = k['code']
                    ws1.cell(row, column=4).value = k['name']
                    ws1.cell(row, column=5).value = h['category']
                    ws1.cell(row, column=6).value = h['code']
                    ws1.cell(row, column=7).value = h['name']
                    row += 1            
            
wb1.save('./data/interim/parsing_3.xlsx')

In [10]:
for i in dct['data']:
    if i['code'][:4] in ['Клас', 'LAB0']:
        for j in i['groups']:
            for g in j['groups']:
                for k in g['services']:
                    print(i['code'])
                    print(j['code'])
                    print(g['code'])
                    print(g['name'])
                    print(k['category'])
                    print(k['code'])
                    print(k['name'])

Клас 20
7 (97)
1999
Рентгеноскопія
imaging
59763-00
Інсуфляцiя повітря під час рентгеноскопії
Клас 20
7 (97)
1999
Рентгеноскопія
imaging
60503-00
Рентгеноскопія
Клас 20
1 (15)
1944
Ультразвукове дуплексне дослідження черепних, каротидних та вертебральних судин
imaging
55274-00
Ультразвукове дуплексне дослідження позачерепних, каротидних та вертебральних судин
Клас 20
1 (15)
1944
Ультразвукове дуплексне дослідження черепних, каротидних та вертебральних судин
imaging
55280-00
Ультразвукове дуплексне дослідження внутрішньочерепних судин
Клас 20
1 (15)
1949
Інше ультразвукове дослідження
imaging
30688-00
Ендоскопічне ультразвукове дослідження
Клас 20
1 (15)
1949
Інше ультразвукове дослідження
imaging
90908-01
Високоінтенсивне сфокусоване ультразвукове дослідження [ВІСУЗД]
Клас 20
1 (15)
1947
Ультразвукове дуплексне дослідження артерії або печеристого тіла статевого члена
imaging
55282-00
Ультразвукове дуплексне дослідження артерії статевого члена
Клас 20
1 (15)
1947
Ультразвукове дуплексне

KeyError: 'groups'

In [20]:
df = pd.read_excel("./data/interim/first.xlsx", sheet_name='Sheet2')
def list_class(s):
    st = set(s)
    return ', '.join(list(st))
df = df.groupby(['request_allowed']).agg({'code':list_class})

In [22]:
df.to_excel("./data/interim/class.xlsx", index=False)